In [ ]:
from langchain_community.llms import Ollama

model = Ollama(model="llama3")
#model

In [2]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain = model | parser

In [3]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader
PDF_loader = PyPDFLoader("mlschool.pdf")
TXT_loader = TextLoader("data_text.txt",encoding="utf-8")

PDF_pages = PDF_loader.load_and_split()
# TXT_pages = TXT_loader.load()

PDF_pages
# TXT_pages

[Document(page_content='Building Machine Learning Systems That Don\'t\nSuck\nA live, interactive program that\'ll help you build production-readymachine\nlearning systems from the ground up.\nNext cohort:\xa0July1 - 18, 2024\nCheck the schedulefor more details about upcoming cohorts.\nI want to join!Sign in\nLearn how to design, build, deploy, and scale machine learning\nsystems to solve real-world problems.\nI\'ll lose my mind if I see another book or course teaching people the same basic\nideas for the hundredth time. Most people are stuck in beginner mode, and finding\nhelp to solve real-world problems is hard.\nI want to change that.\nI started writing software 30 years ago. I\'ve written pipelines and trained models\nfor some of the largest companies in the world. I want to show you how to do the\nsame.\nThis is the class I wish I had taken when I started."This is the best machine learning course I\'ve done.\nWorth every cent."\n— Jose Reyes, AI/ML at Cevo Australia6/24/24, 11:16 

In [4]:
from langchain.prompts import PromptTemplate
template = """
Answer the question based on  the context.If you can't answer the question then relpy "I don't know!"
Context  : {context}
Question : {question}
"""
prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context",question="Here is a Question")

'\nAnswer the question based on  the context.If you can\'t answer the question then relpy "I don\'t know!"\nContext  : Here is some context\nQuestion : Here is a Question\n'

In [5]:
chain = prompt | model | parser

In [14]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [6]:
chain.invoke({
    "context": "My Favorite sport is Cricket",
    "question": "What is my favorite dish?"
})

"I don't know! The context only mentions that your favorite sport is cricket, but it doesn't mention anything about food or a favorite dish. I'm not aware of any information about what you like to eat."

In [16]:
translation_prompt = PromptTemplate.from_template(
    "Translation {answer} to {language}"
)

In [19]:
from operator import itemgetter

translation_chain = (
    {"answer": chain,"language":itemgetter("language")} | translation_prompt | model | parser
)

translation_chain.input_schema.schema()

{'title': 'RunnableParallel<answer,language>Input',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'},
  'language': {'title': 'Language'}}}

In [21]:
translation_chain.invoke(
    {
        "context" : "I have four cars and three bikes.",
        "question": "How many cars I have?",
        "language": "Spainsh"
    }
)

'Tienes 4 coches.\n\nHere\'s a breakdown of the translation:\n\n* "You" is translated as "Tienes", which is the second-person singular form of the verb "tener" (to have).\n* "have" is translated as "tienes".\n* "4" remains unchanged, as it is a number.\n* "cars" is translated as "coches".'

In [7]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model='llama3')

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(PDF_pages,embedding=embeddings)


In [10]:
retriver = vectorstore.as_retriever()
retriver.invoke("Machine Learning")

[Document(page_content="You'll learn best practices to tackle the most significant challenges machine\nlearning engineers face to build, evaluate, run, monitor, and maintain machine\nlearning systems in real-world scenarios.\nYou'll learn how to use techniques like active learning, distributed training,\nadversarial validation, human-in-the-loop deployments, model compression,\ntest-time augmentation, testing in production, among many others.\nYou'll learn how to create training, deploying, monitoring, and inference\npipelines using Amazon SageMaker and open-source tools.\nForget about theoretical concepts. This program will show you some of the things\nI've learned from real-life examples I've built during more than 30 years in the\nindustry.\nCheck the program syllabus\xa0→\nReal-life examples and case studies\nLearn from practical experience building machine\nlearning systems that work in the real world.\nLive, interactive sessions\nAsk questions and interact with the instructor and

In [ ]:
from operator import itemgetter

chain = (
{
    "context": itemgetter("question") | retriver,
    "question": itemgetter("question")
}
| prompt | model | parser )

chain.invoke({"question": "What is Machine Learning?"})

In [12]:
questions =[
    "What is the purpose of the course?",
    "How many hours of live sessions?",
    "How much program cost?"
]

for question in questions:
    print(f"Question:{question}")
    print(f"Answer:{chain.invoke({'question':question})}")

Question:What is the purpose of the course?
Answer:According to the context, the purpose of the course is "to use machine learning to solve real-world problems".
Question:How many hours of live sessions?
Answer:According to the context, there are 2 hours of live sessions every Monday and Thursday.
Question:How much program cost?
Answer:I don't know!


In [ ]:
for s in chain.stream({"question": "What is the purpose of the course?"}):
    print(s,end ="", flush=True)

In [ ]:
# To give the answer at a time in an array
chain.batch([{"question": q} for q in questions])